In [10]:
### imports
import json
from copy import deepcopy
import pandas as pd
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
import re
import regex
tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
stopW = stopwords.words('english')
# stopW = ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
# print(sorted(stopW))
import pandas as pd
import math 

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 

In [11]:
### a method to Read a json file with img captions and article titles and parse it into set of documents

def getDocs(sourcePath):
    medDocs = []
    sourceFile = open(sourcePath, 'r')

    count = 1
    for line in sourceFile:
        medDocs.append(json.loads(line))
        count += 1

    sourceFile.close()
    
    # for k in range(1,count):
    #     print(str(k) + " : " + medDocs[k]['docId'])
    return medDocs


sourcePath = 'assets/medImages.jl'
medDocs = getDocs(sourcePath)
print(medDocs[262]['caption'])
medDocs[262]

Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.


{'articleId': 'PMC7282154',
 'image_path': 'PMC7282154-6_i2376-0605-6-2-e50-f06.jpg',
 'image_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/bin/i2376-0605-6-2-e50-f06.jpg',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/',
 'articleTitle': 'RECURRENT INVASIVE DUCTAL BREAST CARCINOMA PRESENTING AS PRIMARY ADRENAL INSUFFICIENCY WITH ADRENAL CRISIS',
 'caption': 'Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.'}

In [21]:
# method that cleans the title and caption text in each document
# the result is tokenized titles and caption without unwanted brackets, numbers, spaces, tags, unicode charts;
# result includes no punctuation and stopwards and tokens with length less than 2 symbols
# all tokens are lower case


def preprocessContent(content):
    sentences = sent_tokenize(content)
    tokens = []
    for sent in sentences:
        # removes unicode
        cleanedTokens = re.sub(r'[^\x00-\x7F]+', ' ', sent)
        # removes menions
        cleanedTokens = re.sub(r'@\w+', '', cleanedTokens)
        # removes numbers
#         cleanedTokens = re.sub('[0-9]+', '', cleanedTokens)
        # removes nested brackets 
        cleanedTokens = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', cleanedTokens) 
        # removes nested currly brackets
        cleanedTokens = regex.sub('\{(?:[^}{]|\{[^}{]*\})*\}', '', cleanedTokens) 
        # removes html tags
        cleanedTokens = re.sub('\<+/*\w*/*\>+', '', cleanedTokens)
        # removes punctuation
        cleanedTokens = re.sub(r'[%s]' % re.escape(punctuation), ' ', cleanedTokens)
        # removes doubled spaces
        cleanedTokens = re.sub(r'\s{2,}', ' ', cleanedTokens)
        sentTokens = tokenizer.tokenize(cleanedTokens)
        # lower case each token and removes tokens with length less than 2
        sentTokens = [tok.lower() for tok in sentTokens if len(tok) >2]
        # removes stop words 
        sentTokens2 = [tok for tok in sentTokens if not tok.lower() in stopW ]
        tokens += sentTokens2
    return tokens


def cleanDocs(medDocs):
    cleanedDocs = deepcopy(medDocs)
    for i, doc in enumerate(cleanedDocs):
        doc['articleTitle'] = preprocessContent(doc['articleTitle'] )
        doc['caption'] = preprocessContent(doc['caption'])
    return cleanedDocs

cleanedDocs = cleanDocs(medDocs)

In [18]:
print(len(cleanedDocs))
print(medDocs[262]['caption'])
print(cleanedDocs[262]['caption'])
# cleanedDocs[262]

263
Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.
['histologic', 'section', 'adrenal', 'metastatic', 'disease', 'tumor', 'cells', 'arranged', 'solid', 'nests', 'infiltrating', 'lymph', 'node', 'tissue']


In [23]:
def createDocList(corpus):
    titlesList = []
    captionsList = []
    for doc in corpus:
        titlesList.append(doc['articleTitle'])
        captionsList.append(doc['caption'])
    return titlesList, captionsList

titlesList, captionsList = createDocList(medDocs)
print(medDocs[262]['caption'])
print(captionsList[262])
titlesList[262]

Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.
Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.


'RECURRENT INVASIVE DUCTAL BREAST CARCINOMA PRESENTING AS PRIMARY ADRENAL INSUFFICIENCY WITH ADRENAL CRISIS'

In [32]:
def computeTFIDF(docList):
    vectorizer = TfidfVectorizer(lowercase = False, tokenizer = preprocessContent)
    vectors = vectorizer.fit_transform(docList)
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    return denselist, df

title_tfidf, title_df = computeTFIDF(titlesList)
caption_tfidf, caption_df = computeTFIDF(captionsList)
caption_df

,000,001,0125,100,1000,102,105,10th,120,121,...,xq28,year,years,yellow,yielding,young,zannikos,zero,zidovudine,zonula
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [34]:
print(title_tfidf[0])
title_df


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.277356415106499, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

,175,3a4,550,absorption,acquired,additional,adrenal,adults,advanced,advancing,...,vignette,virals,virus,vitro,vivo,whole,women,xenografted,xrcc,young
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.277356,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.277356,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
5,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
6,0.0,0.0,0.0,0.0,0.335523,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
7,0.0,0.0,0.0,0.0,0.335523,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
9,0.0,0.0,0.0,0.0,0.335523,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
